In [ ]:
import heapq
import numpy as np
from icecream import ic
from random import choice
from tqdm.auto import tqdm
from collections import namedtuple

In [ ]:
PUZZLE_DIM = 4
Action = namedtuple('Action', ['pos1', 'pos2'])

In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x_start, y_start = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x_start > 0:
        actions.append(Action((x_start, y_start), (x_start-1, y_start)))
    if x_start < PUZZLE_DIM - 1:
        actions.append(Action((x_start, y_start), (x_start+1, y_start)))
    if y_start > 0:
        actions.append(Action((x_start, y_start), (x_start, y_start-1)))
    if y_start < PUZZLE_DIM - 1:
        actions.append(Action((x_start, y_start), (x_start, y_start+1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [ ]:
def get_correct_pos():
    # row: (n-1) // PUZZLE_DIM, column: n - PUZZLE_DIM * row - 1
    correct_pos = {n : ((n-1) // PUZZLE_DIM, n - PUZZLE_DIM * ((n-1) // PUZZLE_DIM) - 1) for n in range(1, PUZZLE_DIM**2)}
    correct_pos[0] = (PUZZLE_DIM-1, PUZZLE_DIM-1)
    return correct_pos

CORRECT_POS = get_correct_pos()

def manhattan_distance(state: np.ndarray) -> int:
    total_distance = 0
    for n in range(PUZZLE_DIM**2):
        x_current, y_current = [int(_[0]) for _ in np.where(state == n)]
        total_distance += abs(x_current - CORRECT_POS[n][0]) + abs(y_current - CORRECT_POS[n][1])
    return total_distance

In [ ]:
# Two tiles are in a linear conflict if they are in the same row or column, and their target positions are in the same row or column, 
# and the target position of one of the tiles is blocked by the other tile in that row.
def n_linear_conflicts(state: np.ndarray) -> int:
    conflicts = 0

    # row conflicts
    for r in range(PUZZLE_DIM):
        for c in range(PUZZLE_DIM):
            for k in range(c+1, PUZZLE_DIM):
                if(state[r][c] and state[r][k] and CORRECT_POS[state[r][c]][0] == r and 
                CORRECT_POS[state[r][k]][0] == r and CORRECT_POS[state[r][c]][1] > CORRECT_POS[state[r][k]][1]):
                    conflicts += 1

    # column conflicts
    for c in range(PUZZLE_DIM):
        for r in range(PUZZLE_DIM):
            for l in range(r+1, PUZZLE_DIM):
                if(state[r][c] and state[l][c] and CORRECT_POS[state[r][c]][1] == c and 
                CORRECT_POS[state[l][c]][1] == c and CORRECT_POS[state[r][c]][0] > CORRECT_POS[state[l][c]][0]):
                    conflicts += 1
    
    return conflicts

def linear_conflicts(state: np.ndarray) -> int:
    return manhattan_distance(state) + n_linear_conflicts(state)

heuristic = linear_conflicts

In [ ]:
RANDOMIZE_STEPS = 1000
GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
print(GOAL_STATE)
initial_state = GOAL_STATE.copy()
for step in range(RANDOMIZE_STEPS): # use tqdm
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
    
h_distance = heuristic(initial_state)

In [ ]:
class Node:
    def __init__(self, state: np.ndarray, parent, g: int, h: int) -> None:
        self.state = state
        self.parent = parent
        self.g = g
        self.h = h

    def __lt__(self, next):
        return self.g + self.h < next.g + next.h

In [ ]:
frontier = []
explored = set()
heapq.heappush(frontier, Node(initial_state, None, 0, h_distance))

while frontier:
    current_node = heapq.heappop(frontier)
    # print(current_node.state)
    if np.array_equal(current_node.state, GOAL_STATE):
        path = []
        node = current_node
        while node.parent != None:
            path.append(node.state)
            node = node.parent
        path.append(initial_state)
        print(path[::-1])
        break
    for action in available_actions(current_node.state):
        new_state = do_action(current_node.state, action)
        if tuple(new_state.flatten()) not in explored:
            explored.add(tuple(new_state.flatten()))
            heuristic_distance = heuristic(new_state)
            heapq.heappush(frontier, Node(new_state, current_node, current_node.g + 1, heuristic_distance))

print(f"\nSolve {PUZZLE_DIM*PUZZLE_DIM-1}-puzzle in {len(path)} moves.")
